# lets get some RF + PCAs going

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
CARIACO = pd.read_csv("../DATA/January/Combined_CARIACO_data_v5.csv")
CARIACO['date'] = pd.to_datetime(CARIACO['time_month'])
CARIACO = CARIACO.set_index('date')

In [6]:
CARIACO.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 316 entries, 1995-11-01 to 2021-11-01
Data columns (total 96 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    316 non-null    int64  
 1   cdww                          299 non-null    float64
 2   iews                          299 non-null    float64
 3   inss                          299 non-null    float64
 4   mcc                           299 non-null    float64
 5   mwp                           299 non-null    float64
 6   si10                          299 non-null    float64
 7   sp                            300 non-null    float64
 8   sst                           299 non-null    float64
 9   t2m                           299 non-null    float64
 10  tauoc                         299 non-null    float64
 11  u10                           299 non-null    float64
 12  ust                           300 non-null   

## Create Subframes without NAs

In [34]:
CSUB1 = CARIACO[['u10',
                 'sst',
                 'mtpr',
                 'cdir',
                 'e',
                 'Satellite_chla',
                 'Isotherm_21',
                 'Temperature_35m',
                 'Chlorophyll',
                 'Tchla'
        ]].dropna()

CSUB1.columns

Index(['u10', 'sst', 'mtpr', 'cdir', 'e', 'Satellite_chla', 'Isotherm_21',
       'Temperature_35m', 'Chlorophyll', 'Tchla'],
      dtype='object')

# OH SHIT!

In [35]:
import pandas as pd
import numpy as np
from tslearn.clustering import TimeSeriesKMeans, KShape, KernelKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
#from netdata_pandas.data import get_data, get_chart_list
from am4894plots.plots import plot_lines, plot_lines_grid

In [44]:
# inputs
#host = 'london.my-netdata.io' # host running netdata that we want to pull data from
#n = 60*5 # how many seconds of most recent data to pull
#n_charts = None # If None then pull data for all charts otherwise sample n_charts randomly
n_clusters = 7 # number of clusters to fit
diff = False # take diffs of the data or not
preprocessing_meanvar = True # True to use TimeSeriesScalerMeanVariance preprocessing
#smooth_n = 15 # n observations to smooth over
#smooth_func = 'mean' # one of ['mean','min','max','sum']
#norm = False # normalize the data to 0-1 range
model = 'kmeans' # one of ['kmeans','kshape','kernelkmeans','dtw']

In [45]:
df = CSUB1

In [46]:
X = df.transpose().values#.transpose().values

if preprocessing_meanvar:
    X = TimeSeriesScalerMeanVariance().fit_transform(X)
    df = pd.DataFrame(X.reshape(df.shape), columns=df.columns, index=df.index)
if model == 'kshape':
    model = KShape(n_clusters=n_clusters, max_iter=10, n_init=2).fit(X)
elif model == 'kmeans':
    model = TimeSeriesKMeans(n_clusters=n_clusters, metric="euclidean", 
                             max_iter=10, n_init=2).fit(X)
elif model == 'dtw':
    model = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", 
                             max_iter=5, n_init=2).fit(X)
elif model == 'kernelkmeans':
    model = KernelKMeans(n_clusters=n_clusters, kernel="gak", 
                         max_iter=5, n_init=2).fit(X)
else:
    model = TimeSeriesKMeans(n_clusters=n_clusters, metric="euclidean", 
                             max_iter=10, n_init=2).fit(X)

In [47]:
X

array([[[-0.47855945],
        [ 0.34348194],
        [ 0.55976618],
        ...,
        [-0.01139708],
        [ 1.36602812],
        [ 0.49860671]],

       [[ 1.41657588],
        [ 0.25924791],
        [-0.71281159],
        ...,
        [ 0.06173515],
        [ 0.65118668],
        [-0.80961068]],

       [[ 0.10915743],
        [-0.69709465],
        [-1.07030909],
        ...,
        [ 1.21777619],
        [-0.04175731],
        [-0.80968188]],

       ...,

       [[ 0.87685884],
        [ 0.42602387],
        [-1.0736123 ],
        ...,
        [ 0.4571696 ],
        [ 0.7910731 ],
        [-0.46742368]],

       [[-0.01846287],
        [-0.42940106],
        [ 0.55403848],
        ...,
        [-0.52176912],
        [-0.2354004 ],
        [ 0.29931733]],

       [[-0.52738545],
        [-0.55313411],
        [-0.18020292],
        ...,
        [ 0.17829771],
        [-0.13040331],
        [ 0.41258225]]])

In [48]:
# build helper df to map metrics to their cluster labels
df_cluster = pd.DataFrame(list(zip(df.columns, model.labels_)), columns=['metric', 'cluster'])

# make some helper dictionaries and lists
cluster_metrics_dict = df_cluster.groupby(['cluster'])['metric'].apply(lambda x: [x for x in x]).to_dict()
cluster_len_dict = df_cluster['cluster'].value_counts().to_dict()
clusters_dropped = [cluster for cluster in cluster_len_dict if cluster_len_dict[cluster]==1]
clusters_final = [cluster for cluster in cluster_len_dict if cluster_len_dict[cluster]>1]
clusters_final.sort()

df_cluster.head()

,metric,cluster
0,u10,3
1,sst,1
2,mtpr,5
3,cdir,0
4,e,4


In [49]:
cluster_len_dict

{1: 3, 2: 2, 3: 1, 5: 1, 0: 1, 4: 1, 6: 1}

In [50]:
import plotly
plotly.offline.init_notebook_mode(connected=True)

In [51]:
for cluster_number in clusters_final:
    # get a rough quality score based on the correlation between metrics in the cluster
    x_corr = df[cluster_metrics_dict[cluster_number]].corr().abs().values
    x_corr_mean = round(x_corr[np.triu_indices(x_corr.shape[0],1)].mean(),2)
    # plot each cluster
    plot_title = f'cluster {cluster_number} (quality={x_corr_mean}, n={cluster_len_dict[cluster_number]})'
    plot_lines(df, cols=cluster_metrics_dict[cluster_number], renderer='colab', theme=None, title=plot_title)